In [ ]:
import torch
import torch.nn.functional as F

class VPLoss:
    def __call__(self, model, x_0, t, noise):
        x_t = self.q_sample(x_0, t, noise)
        pred_noise = model(x_t, t)
        return F.mse_loss(pred_noise, noise) # recheck the loss function

    def q_sample(self, x_0, t, noise):
        # Apply forward process to add noise
        # return x_t
        pass
